In [1]:
%load_ext autoreload
%autoreload 2
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchtext import data, datasets
import spacy
from matplotlib import pyplot as plt
import numpy as np
from tqdm.notebook import tqdm

dev = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
from utils import Multi30KEn2DeDatasetTokenizer
dataset = Multi30KEn2DeDatasetTokenizer(dev=dev)

In [3]:
dl = DataLoader(
    dataset.train, shuffle=True, batch_size=128, collate_fn=dataset.collate_fn)
example =  next(iter(dl))
{a: example[a].shape for a in example}

{'src': torch.Size([128, 31]),
 'trg': torch.Size([128, 26]),
 'src_mask': torch.Size([128, 1, 1, 31]),
 'trg_mask': torch.Size([128, 1, 26, 26]),
 'trg_y': torch.Size([128, 26])}

In [4]:
from transformers.model import EncoderDecoder, Trainer
import os

def get_trainer():
    model = EncoderDecoder(
        len(dataset.src_vocab),
        len(dataset.trg_vocab)).to(dev)

    trainer = Trainer(model)
    try:
        chks = os.listdir('./chkpnts')
        ns = [int(chk.split('checkpnt_step-')[1].split('k.pt')[0])
             for chk in chks]
        if len(ns) == 0:
            raise Exception('No checkpoints in ./chkpnts')
        n = sorted(ns)[-1]
        print(f'checkpnt_epoch-{n}k.pt')
        trainer.load(f'chkpnts/checkpnt_step-{n}k.pt')

    except Exception as e:
        print('error: ', e)
    return trainer, model

trainer, model = get_trainer()

error:  No checkpoints in ./chkpnts


In [5]:
trainer.train_loop(1_000_000, dl)

  0%|          | 0/1000000 [00:00<?, ?it/s]


KeyboardInterrupt



In [ ]:
src = example['src']
tgt = example['trg']
src_mask = example['src_mask']
tgt_mask = example['trg_mask']
trg_y = example['trg_y']

output = model(src, tgt, src_mask, tgt_mask)
phrase = torch.argmax(output[0], dim=-1)
dataset.itos(phrase)